In [21]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split


In [22]:
def load_images(path):
    data = []
    for file in os.listdir(path):
        if file.endswith(('jpg', 'jpeg', 'png')):
            img = cv2.imread(os.path.join(path, file))
            img = cv2.resize(img, (128, 128))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            data.append(img / 255.0)
    return np.array(data)


In [23]:
print("🚀 Loading dataset...")
low_dir = 'lol_dataset/our485/low'
high_dir = 'lol_dataset/our485/high'
low_imgs = load_images(low_dir)
high_imgs = load_images(high_dir)
print(f"Loaded {len(low_imgs)} low-light images and {len(high_imgs)} high-light images.")


🚀 Loading dataset...
Loaded 485 low-light images and 485 high-light images.


In [24]:
train_low, val_low, train_high, val_high = train_test_split(low_imgs, high_imgs, test_size=0.1, random_state=42)
print(f"Training data: {len(train_low)} images | Validation data: {len(val_low)} images")


Training data: 436 images | Validation data: 49 images


In [30]:
def build_enhancer():
    inp = Input(shape=(128, 128, 3))  # Change to (128, 128, 3)

    c1 = Conv2D(64, (3,3), activation='relu', padding='same')(inp)
    p1 = MaxPooling2D((2,2))(c1)

    c2 = Conv2D(128, (3,3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2,2))(c2)

    c3 = Conv2D(256, (3,3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2,2))(c3)

    bn = Conv2D(512, (3,3), activation='relu', padding='same')(p3)

    u3 = UpSampling2D((2,2))(bn)
    m3 = Concatenate()([u3, c3])
    c4 = Conv2D(256, (3,3), activation='relu', padding='same')(m3)

    u2 = UpSampling2D((2,2))(c4)
    m2 = Concatenate()([u2, c2])
    c5 = Conv2D(128, (3,3), activation='relu', padding='same')(m2)

    u1 = UpSampling2D((2,2))(c5)
    m1 = Concatenate()([u1, c1])
    c6 = Conv2D(64, (3,3), activation='relu', padding='same')(m1)

    out = Conv2D(3, (1,1), activation='sigmoid')(c6)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae', psnr, ssim, accuracy])
    return model


In [31]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [32]:
model = build_enhancer()
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 128, 128,  │      1,792 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 64, 64,    │          0 │ conv2d_32[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_12… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 32, 32,    │          0 │ conv2d_33[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_13… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 16, 16,    │          0 │ conv2d_34[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_14… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_12    │ (None, 32, 32,    │          0 │ conv2d_35[0][0]   │
│ (UpSampling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, 32, 32,    │          0 │ up_sampling2d_12… │
│ (Concatenate)       │ 768)              │            │ conv2d_34[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, 32, 32,    │  1,769,728 │ concatenate_12[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_13    │ (None, 64, 64,    │          0 │ conv2d_36[0][0]   │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_13      │ (None, 64, 64,    │          0 │ up_sampling2d_13… │
│ (Concatenate)       │ 384)              │            │ conv2d_33[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_37 (Conv2D)  │ (None, 64, 64,    │    442,496 │ concatenate_13[0… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_14    │ (None, 128, 128,  │          0 │ conv2d_37[0][0]   │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_14      │ (None, 128, 128,  │          0 │ up_sampling2d_14… │
│ (Concatenate)       │ 192)              │            │ conv2d_32[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 128, 128,  │    110,656 │ concatenate_14[0

 Total params: 3,874,051 (14.78 MB)

 Trainable params: 3,874,051 (14.78 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
print("📚 Training model...")
history = model.fit(train_low, train_high, validation_data=(val_low, val_high), epochs=10, batch_size=16)
print("✅ Training complete.")


📚 Training model...
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 197s 7s/step - accuracy: 0.3186 - loss: 0.0468 - mae: 0.1785 - psnr: 13.8028 - ssim: 0.3851 - val_accuracy: 0.3519 - val_loss: 0.0343 - val_mae: 0.1568 - val_psnr: 15.4006 - val_ssim: 0.5486
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 182s 7s/step - accuracy: 0.4052 - loss: 0.0301 - mae: 0.1412 - psnr: 16.1864 - ssim: 0.5566 - val_accuracy: 0.4297 - val_loss: 0.0280 - val_mae: 0.1396 - val_psnr: 16.3195 - val_ssim: 0.6443
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 178s 6s/step - accuracy: 0.4463 - loss: 0.0253 - mae: 0.1288 - psnr: 16.9811 - ssim: 0.6361 - val_accuracy: 0.4066 - val_loss: 0.0432 - val_mae: 0.1661 - val_psnr: 14.4710 - val_ssim: 0.6114
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 179s 6s/step - accuracy: 0.4197 - loss: 0.0296 - mae: 0.1386 - psnr: 16.4645 - ssim: 0.6417 - val_accuracy: 0.4790 - val_loss: 0.0261 - val_mae: 0.1331 - val_psnr: 17.1412 - val_ssim: 0.7119
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 178s 6s/step - accuracy: 0

In [35]:
import matplotlib.pyplot as plt

def visualize_prediction(model, low_input, high_target, index=0):
    # Predicting the high-light image from low-light input
    predicted = model.predict(low_input[index:index+1])[0]

    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(low_input[index])  # Low-light image
    axes[0].set_title('Low-light Input')
    axes[1].imshow(high_target[index])  # High-light target
    axes[1].set_title('High-light Target')
    axes[2].imshow(predicted)  # Predicted image
    axes[2].set_title('Predicted High-light Image')
    plt.show()
